<a href="https://colab.research.google.com/github/ansfarooq7/l4-project/blob/main/prototypes/L4_Project_second.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install pronouncing
!pip install wikipedia
import torch
import pronouncing
import wikipedia
import re
import random

     |████████████████████████████████| 3.1 MB 12.0 MB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 59 kB 5.8 MB/s 
     |████████████████████████████████| 895 kB 55.1 MB/s 
     |████████████████████████████████| 3.3 MB 43.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 939 kB 12.7 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=8bfe9404af06ab71c099a0e0de30dd74c44034a93c9b58358e698079eef45525
  Stored in directory: /root/.cache/pip/wheels/09/e8/c0/3606d42fdbf5f3871564eb6a353591a8f5deeed013fdb73921
Successfully built pronouncing
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11696 sha256=b9f747f6afd4302122ec39dbf95c5b5e77ea31204acf70e40f9871e7f556ea84
  Stored in directory: /root/.cache/pip/wheels/1

In [2]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForMaskedLM.from_pretrained('roberta-base')

def set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).

    Args:
        seed (:obj:`int`): The seed to set.
    """
    #random.seed(seed)
    #np.random.seed(seed)
    #if is_torch_available():
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
    #if is_tf_available():
        #tf.random.set_seed(seed)
    
def get_prediction(sent):
    
    token_ids = tokenizer.encode(sent, return_tensors='pt')
    masked_position = (token_ids.squeeze() == tokenizer.mask_token_id).nonzero()
    masked_pos = [mask.item() for mask in masked_position ]

    with torch.no_grad():
        output = model(token_ids)

    last_hidden_state = output[0].squeeze()

    list_of_list =[]
    for index,mask_index in enumerate(masked_pos):
        mask_hidden_state = last_hidden_state[mask_index]
        idx = torch.topk(mask_hidden_state, k=5, dim=0)[1]
        words = [tokenizer.decode(i.item()).strip() for i in idx]
        list_of_list.append(words)
        print("Mask ",index+1,"Guesses : ",words)
    
    best_guess = ""
    for j in list_of_list:
        best_guess = best_guess+" "+j[0]
        
    return best_guess

sentence = "Manchester United are ___ ___ ___ champions."
print ("Original Sentence: ",sentence)
sentence = sentence.replace("___","<mask>")
print ("Original Sentence replaced with mask: ",sentence)
print ("\n")

predicted_blanks = get_prediction(sentence)
print ("\nBest guess for fill in the blank :::",predicted_blanks)

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Original Sentence:  Manchester United are ___ ___ ___ champions.
Original Sentence replaced with mask:  Manchester United are <mask> <mask> <mask> champions.


Mask  1 Guesses :  ['the', 'defending', 'currently', 'reigning', 'crowned']
Mask  2 Guesses :  ['reigning', 'defending', 'the', 'crowned', 'Premier']
Mask  3 Guesses :  ['League', 'league', 'defending', 'world', 'four']

Best guess for fill in the blank :::  the reigning League


In [3]:
from transformers import pipeline
text_generation = pipeline("text-generation")
set_seed(31)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [4]:
pronouncing.rhymes("with") 

[]

In [5]:
from transformers import GPT2TokenizerFast

def get_inputs_length(input):
  gpt2_tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
  input_ids = gpt2_tokenizer(input)['input_ids']
  return len(input_ids)

In [9]:
from transformers import pipeline
text_generation = pipeline("text-generation")

limericks = []

topic = input("Please enter a topic: ")
topic_summary = re.sub(r'[^\w\s]', '', wikipedia.summary(topic))
# if len(topic_summary) > 2000:
#   topic_summary = topic_summary[:2000]
word_list = topic_summary.split()
topic_summary_len = len(topic_summary)
no_of_words = len(word_list)
inputs_len = get_inputs_length(topic_summary)
print(f"Topic Summary: {topic_summary}")
print(f"Topic Summary Length: {topic_summary_len}")
print(f"No of Words in Summary: {no_of_words}")
print(f"Length of Input IDs: {inputs_len}")

# starting_words = [["That", "Had", "Not", "But", "That"], 
#                   ["There", "Who", "She", "Tormenting", "Til"],
#                   ["Relentless", "This", "First", "and", "then"],
#                   ["There", "Who", "That", "To", "She"],
#                   ["There", "Who", "Two", "Four", "Have"]]

starting_words = ["That", "Had", "Not", "But", "With", "I", "Because", "There", "Who", "She", "He", "To", "Whose", "In", "And", "When", "Or", "So", "The", "Of", "Every", "Whom"]

# rhyming_words = [["told", "bold", "woodchuck", "truck", "road"], 
#                  ["Nice", "grease", "house", "spouse", "peace"],
#                  ["deadlines", "lines", "edits", "credits", "wine"],
#                  ["Lynn", "thin", "essayed", "lemonade", "in"],
#                  ["beard", "feared", "hen", "wren", "beard"]]                 

for i in range(5):
  rhyming_words = []
  while len(rhyming_words) < 5:
    first_line = text_generation(topic_summary + " " + random.choice(starting_words), max_length=inputs_len + 9, do_sample=False, return_full_text=False)[0]['generated_text']
    print(f"First Line: {first_line}")
    rhyming_words = pronouncing.rhymes(first_line.split()[-1]) 
    print(rhyming_words)
    limerick = first_line + "\n"

  for j in range(4):
    #gpt2_sentence = text_generation(topic_summary + " " + starting_words[i][j], max_length=no_of_words + 4, do_sample=False)[0]
    gpt2_sentence = text_generation(topic_summary + " " + random.choice(starting_words), max_length=inputs_len + 3, do_sample=False, return_full_text=False)[0]
    #sentence = gpt2_sentence['generated_text'] + " ___ ___ ___ " + rhyming_words[i][j]
    sentence = gpt2_sentence['generated_text'] + " ___ ___ ___ " + rhyming_words[j]
    print("Original Sentence: ", sentence)
    sentence = sentence.replace("___","<mask>")
    print("Original Sentence replaced with mask: ", sentence)
    print("\n")
 
    predicted_blanks = get_prediction(sentence)
    print("\nBest guess for fill in the blank: ", predicted_blanks)
    limerick = limerick + gpt2_sentence['generated_text'] + predicted_blanks + " " + rhyming_words[j] + "\n"

  limericks.append(limerick)

print("\n")
print(f"Generated {len(limericks)} limericks: \n")
for limerick in limericks:
  print(limerick)

No model was supplied, defaulted to gpt2 (https://huggingface.co/gpt2)


Please enter a topic: Association football


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Topic Summary: Association football more commonly known as simply football or soccer is a team sport played with a spherical ball between two teams of 11 players It is played by approximately 250 million players in over 200 countries and dependencies making it the worlds most popular sport The game is played on a rectangular field called a pitch with a goal at each end The object of the game is to score more goals than the opposition by moving the ball beyond the goal line into the opposing goal usually within a time frame of 90 or more minutes
Football is played in accordance with a set of rules known as the Laws of the Game The ball is 6870 cm 2728 in in circumference and known as the football The two teams compete to get the ball into the other teams goal between the posts and under the bar thereby scoring a goal Players are not allowed to touch the ball with hands or arms while it is in play except for the goalkeepers within the penalty area Players may use any other part of their 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Mask  1 Guesses :  ['world', 'same', 'time', 'day', 'year']
Mask  2 Guesses :  [',', '</s>', 'that', 'and', 'of']
Mask  3 Guesses :  ['here', 'as', 'Here', 'how', ',']

Best guess for fill in the blank:   world , here
Original Sentence:  aded by ___ ___ ___ biz
Original Sentence replaced with mask:  aded by <mask> <mask> <mask> biz




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Mask  1 Guesses :  [':', 'the', '</s>', 'j', 'r']
Mask  2 Guesses :  ['</s>', 'in', ':', 'from', 'at']
Mask  3 Guesses :  ['the', ',', 'iz', 'biz', 'in']

Best guess for fill in the blank:   : </s> the
Original Sentence:   the exception ___ ___ ___ buis
Original Sentence replaced with mask:   the exception <mask> <mask> <mask> buis




Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Mask  1 Guesses :  ['to', ':', 'of', 'is', '</s>']
Mask  2 Guesses :  ['the', 'le', 'c', 'de', 'la']
Mask  3 Guesses :  ['les', 'is', 'de', 'des', 'le']

Best guess for fill in the blank:   to the les
Original Sentence:   far the ___ ___ ___ chandeliers
Original Sentence replaced with mask:   far the <mask> <mask> <mask> chandeliers


Mask  1 Guesses :  ['world', 'light', 'rise', 'glass', 'power']
Mask  2 Guesses :  ['of', ':', '-', 'and', ',']
Mask  3 Guesses :  ['the', 'of', 'and', 'crystal', 'glass']

Best guess for fill in the blank:   world of the


Generated 5 limericks: 

 the world's top 10 mens leagues
 the world 's the military fatigues
 far the politics of the intrigues
 is the top of the league's
 the World Cup ' top leagues'

 be eligible for the 2018 FIFA World Cup
ets of the </s> of bupp
 the exception is the : chupp
 the exception to the : closeup
 of the cup of the cup

 only is the world's largest football league
 are over the </s> the creagh
 the exception of the the